In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor
from math import sqrt
df = pd.read_csv("./desktop/AIRB_NYC_2019.csv")
df['neighbourhood_group'].replace(('Brooklyn', 'Manhattan','Queens','Staten Island','Bronx'), (1,2,3,4,5), inplace=True)
k=df['id']
df['room_type'].replace(('Private room', 'Entire home/apt','Shared room'), (1,2,3), inplace=True)
X=df.drop(['id','name','host_id','host_name','last_review','price','neighbourhood',],axis=1)
X['reviews_per_month'] = X['reviews_per_month'].fillna(0)
#y1=df['price']
from sklearn import preprocessing
scaler  = preprocessing.StandardScaler()
X_scaled = scaler.fit(X.values)
X= X_scaled.transform(X.values)
print(k)

df=df.drop(['id','name','host_id','host_name','last_review','neighbourhood'],axis=1)

0            2539
1            2595
2            3647
3            3831
4            5022
           ...   
48890    36484665
48891    36485057
48892    36485431
48893    36485609
48894    36487245
Name: id, Length: 48895, dtype: int64


In [2]:
df.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,1,40.64749,-73.97237,1,149,1,9,0.21,6,365
1,2,40.75362,-73.98377,2,225,1,45,0.38,2,355
2,2,40.80902,-73.94190,1,150,3,0,NaN,1,365
3,1,40.68514,-73.95976,2,89,1,270,4.64,1,194
4,2,40.79851,-73.94399,2,80,10,9,0.10,1,0


In [3]:
#df=df.drop(['id','name','host_id','host_name','last_review','neighbourhood'],axis=1)
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
from sklearn import preprocessing
scaler  = preprocessing.StandardScaler()
df_scaled = scaler.fit(df)
df1=df_scaled.transform(df)
df1=pd.DataFrame(df1)
y=df1[4]
y=y.values
X.shape
y.shape








(48895,)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Dimensions of the training feature matrix: {}'.format(X_train.shape))
print('Dimensions of the training target vector: {}'.format(y_train.shape))
print('Dimensions of the test feature matrix: {}'.format(X_test.shape))
print('Dimensions of the test target vector: {}'.format(y_test.shape))


Dimensions of the training feature matrix: (39116, 9)
Dimensions of the training target vector: (39116,)
Dimensions of the test feature matrix: (9779, 9)
Dimensions of the test target vector: (9779,)


In [5]:
model = XGBRegressor()              
model.fit(X_train,y_train)            
y_pred = model.predict(X_test)
y_pred.shape
print(y_pred)
print(y_test)
y_pred.shape


#max_depth=5, learning_rate=0.1, n_estimators=160, silent=True, objective='multi:softmax'

[12:28:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[ 0.02953801 -0.4774688  -0.1424442  ... -0.35279745  0.43230358
  0.3355024 ]
[-0.26533513 -0.51101316 -0.13625007 ... -0.34445179  1.44608301
 -0.1154299 ]


(9779,)

In [6]:
model.score(X_test, y_test)


0.1592232637015356

In [7]:
k.tolist()



[2539,
 2595,
 3647,
 3831,
 5022,
 5099,
 5121,
 5178,
 5203,
 5238,
 5295,
 5441,
 5803,
 6021,
 6090,
 6848,
 7097,
 7322,
 7726,
 7750,
 7801,
 8024,
 8025,
 8110,
 8490,
 8505,
 8700,
 9357,
 9518,
 9657,
 9668,
 9704,
 9782,
 9783,
 10452,
 10962,
 11452,
 11708,
 11943,
 12048,
 12192,
 12299,
 12303,
 12318,
 12343,
 12627,
 12937,
 12940,
 13050,
 13394,
 13808,
 14287,
 14290,
 14314,
 14322,
 14377,
 14991,
 15220,
 15338,
 15341,
 15385,
 15396,
 15711,
 16326,
 16338,
 16421,
 16458,
 16580,
 16595,
 16821,
 16974,
 17037,
 17092,
 17693,
 17747,
 18127,
 18152,
 18198,
 18590,
 18728,
 18764,
 19159,
 19169,
 19282,
 19319,
 19601,
 19812,
 20299,
 20300,
 20611,
 20724,
 20734,
 20755,
 20853,
 20913,
 21293,
 21456,
 21644,
 21794,
 22911,
 22918,
 23135,
 23501,
 23686,
 24143,
 24285,
 25235,
 25406,
 25696,
 26012,
 26362,
 26520,
 26559,
 26785,
 26933,
 26954,
 26969,
 27006,
 27385,
 27531,
 27644,
 27659,
 27759,
 27883,
 28321,
 28396,
 28907,
 29012,
 29013,
 2

In [8]:
from bs4 import BeautifulSoup
import time
room_photo=[]
room_photo1=[]
import requests
for room_id in range(len(k.tolist())):
    start_url_nyc = "https://www.airbnb.com/rooms/"
    start_url_nyc=start_url_nyc + str(k.tolist()[room_id])
    rsp = requests.get(start_url_nyc)
    soup = BeautifulSoup(rsp.text, 'html.parser')
    item_list = soup.find(attrs={"class":"with-new-header"})
    main_list = item_list.find(attrs={"id":"site-content"})
    image_list = main_list.find_all("img")
    for image1 in image_list:
        image1_url=image1.get('src')
        #print(image1_url)
        room_photo.append(image1_url)
        time.sleep(1)
    room_photo1.append(room_photo)
    room_photo=[]
room_photo1=array(room_photo1)
print(room_photo1.shape)
    
print('5')

ConnectionError: HTTPSConnectionPool(host='www.airbnb.com', port=443): Max retries exceeded with url: /rooms/27914324 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a2a38abd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
import keras
x_train=[]
x_test=[]


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import losses
room_photo_train, room_photo_test, y_train, y_test = train_test_split(room_photo1, y, test_size=0.2, random_state=42)
model = VGG16(weights='imagenet', include_top=False)
#img_path = 'elephant.jpg'
for n in range(len(room_photo_train)):
    img =image.load_img(room_photo_test[n], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x_train.append(x)
for n in range(len(room_photo_test)):
    img_test =image.load_img(room_photo_test[n], target_size=(224, 224))
    z = image.img_to_array(img_test)
    z = np.expand_dims(z, axis=0)
    z = preprocess_input(z)
    x_test.append(z)
model.fit(x_train,y_train) 
model.compile(loss=losses.mean_squared_error, optimizer='sgd')
y_pred = model.predict(x_test)
keras.losses.mean_squared_error(y_test, y_pred)